<div class="alert alert-info">
<u><strong>Authors:</strong></u> <b>Alberto Vavassori</b> (alberto.vavassori@polimi.it), <b>Emanuele Capizzi</b> (emanuele.capizzi@mail.polimi.it) - 2024 - Politecnico di Milano, Italy <br>
Developed within the LCZ-ODC project, funded by the Italian Space Agency (agreement n. 2022-30-HH.0).
</div>

# Pan-sharpening of Sentinel-2 imagery with PRISMA panchromatic band

### Introduction

This Notebook contains examples of **pan-sharpening methods applied to multispectral Sentinel-2 images** ([Loncan et al. 2015<sup>1</sup>](#1)), using the panchromatic band of the closest (in time) PRISMA image. Thus, super-resolution is achieved through the fusion of Sentinel-2 and PRISMA images.

Pansharpening methods are implemented in the `methods.py` file. Functions are adapted to the PRISMA/Sentinel-2 imagery context from a dedicated GitHub Repository[<sup>2</sup>](#2).

#### Sentinel-2 data specifications

Information about Sentinel-2 specifications can be found in the mission overview website[<sup>3</sup>](#3).

Specifically, Sentinel-2 carries an optical instrument payload that samples 13 spectral bands in the range 400 - 2400 nm.

| Band | Cube | Central wavelength [nm] | Spatial Resolution [m] |
| :---: | :---: | :----: | :---: |
| B1 | VNIR | 443 | 60 |
| B2 | VNIR | 490 | 10 |
| B3 | VNIR | 560 | 10 |
| B4 | VNIR | 665 | 10 |
| B5 | VNIR | 705 | 20 |
| B6 | VNIR | 740 | 20 |
| B7 | VNIR | 783 | 20 |
| B8a | VNIR/SWIR | 865 | 20 |
| B9 | VNIR/SWIR | 940 | 60 |
| B11 | SWIR | 1610 | 20 |
| B12 | SWIR | 2190 | 20 |


#### PRISMA data specifications

Information about the spectral ranges of the hyperspectral and panchromatic data is available within the PRISMA product specification document[<sup>4</sup>](#4).

Specifically, spatial and spectral resolutions of the Hyperspectral (HS) - including Visible Near Infrared (VNIR) and Short Wave Infrared (SWIR) - and Panchromatic (PAN) bands are the following:

| Sensor | Cube | Spectral Range [nm] | Spatial Resolution [m] | #Bands |
| :---: | :---: | :----: | :---: | :---: |
| HS | VNIR | 400 - 700 | 30 | 66 |
| HS | SWIR | 920 - 2505 | 30 | 171 |
| PAN | PAN | 400 - 700 | 5 | 1 |

### Resources
    
<span id="1">[<sup>1</sup>Loncan, L. et al. «Hyperspectral Pansharpening: A Review». *IEEE Geoscience and Remote Sensing Magazine* 2015, 3(3), 1879–1900. doi: 10.1109/MGRS.2015.2440094](https://ieeexplore.ieee.org/document/7284770)</span><br>
<span id="2">[<sup>2</sup>GitHub Repo for multispectral imagery pansharpening](https://github.com/codegaj/py_pansharpening)</span><br>
<span id="3">[<sup>3</sup>Sentinel-2 Mission Overview](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2/overview)</span><br>
<span id="4">[<sup>4</sup>PRISMA Product Specifications](http://prisma.asi.it/missionselect/docs/PRISMA%20Product%20Specifications_Is2_3.pdf)</span><br>

### <a id='TOC_TOP'></a>Notebook content

</div>
    
 1. [Libraries and Data Preparation](#sec1)
 2. [Component Substitution Based Pansharpening](#sec2)  

<hr>

<div class="alert alert-info" role="alert">
    
## <a id='sec1'></a>&#x27A4; 1. Libraries and Data Preparation

[Back to top](#TOC_TOP)
    
</div>

### Import useful libraries

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Make sure you have the following libraries installed in your working environment:
</div>

- `h5py`
- `sklearn`
- `ipyleaflet`
- `ipywidgets`
- `opencv` (i.e. `cv2`) -> install with `pip install opencv-python`
- `leafmap` -> install with `pip install -U leafmap`

Common libraries are also required (e.g. `numpy`, `rasterio`, `matplotlib`, `pandas`, `geopandas`, `shapely`).

In [ ]:
import numpy as np
import rasterio as rio
from rasterio.crs import CRS
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.plot import show_hist
from rasterio.warp import reproject, Resampling
import matplotlib.pyplot as plt
import matplotlib.image
import ipywidgets as widgets
from sklearn.decomposition import PCA
import geopandas as gpd
import h5py
import pandas as pd
import pyproj
import cv2
from shapely.geometry import Polygon
from rasterio import mask
from shapely.geometry import box
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, LayersControl, Rectangle
import leafmap

In [ ]:
# Import functions and set auto-reload
from methods import *
from metrics import *
%load_ext autoreload
%autoreload 2

### Date selection

Here it is possible to select the Sentinel-2 and PRISMA image acquisition dates. Pansharpening will be performed only to the selected Sentinel-2 image, using the selected PRISMA panchromatic band.

Import the PRISMA image and extract its extent:

In [ ]:
date_prisma_w = widgets.Dropdown(
    options = ['2023-02-09', '2023-03-22', '2023-04-08', '2023-06-17', '2023-07-10', '2023-08-08'],
    value = '2023-02-09',
    description = 'PRISMA date:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
date_prisma_w

In [ ]:
sel_prisma_date = date_prisma_w.value
print(f"The selected date is {sel_prisma_date}.")

In [ ]:
prisma_path = 'coregistered/' + sel_prisma_date.replace('-', '') + '/'
prisma_path

Import the Sentinel-2 image and extract its extent:

In [ ]:
date_s2_w = widgets.Dropdown(
    options = ['2023-02-10', '2023-03-22', '2023-04-26', '2023-06-25', '2023-07-10', '2023-08-19'],
    value = '2023-02-10',
    description = 'Sentinel-2 date:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
date_s2_w

In [ ]:
sel_s2_date = date_s2_w.value
print(f"The selected date is {sel_s2_date}.")

### Selection of the area of interest (AOI)

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Pansharpening will only be carried out in the selected area. 

**Note that, considering the limited number of Sentinel-2 bands in comparison with PRISMA, pansharpening can be carried out in a larger area of interest**.
</div>

Import the polygon representing the area of interest. (A new polygon can also be created)

In [ ]:
polygon = gpd.read_file('./coregistered/' + sel_prisma_date.replace('-', '') + '/aoi_s2.gpkg')

### Import the (Sentinel-2) MS and (PRISMA) PAN bands and clip to the AOI

The co-registered bands of the HS (VNIR) and PAN cubes are imported and clipped to the selected AOI.

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Only the bands of the VNIR are used for pansharpening, since they cover the same region of the elecromagnetic spectrum as the PAN band. Accordingly, the SWIR bands will not be sharpened to the PAN resolution.
</div>

Set the paths of the panchromatic PRISMA band `pan_full` and multispectral Sentinel-2 bands `hs_full` (coregistered).

In [ ]:
pan_full = prisma_path + 'PR_pan_' + sel_prisma_date.replace('-', '') + '_5m.tif'
hs_full = prisma_path + 'S2_' + sel_s2_date.replace('-', '') + '_20m_all_bands_clip.tif'

Set the path and name of the same images clipped to the selected AOI (`pan_path` and `hs_path`).

In [ ]:
pan_path = prisma_path + 'pan_s2.tif'
hs_path = prisma_path + 'hs_s2.tif'

Call the function `clip_pan_hs` defined in `methods.py` to clip the Sentinel-2 and PAN bands to the AOI and save them to GeoTIFF files.

In [ ]:
clip_pan_hs(hs_full, pan_full, polygon, hs_path, pan_path)

<div class="alert alert-success" role="alert">
<span>&#x2714;</span>
<a id='libraries'></a>
The VNIR and PAN bands have been clipped to the AOI and saved to GeoTIFF files.
</div>

### Image preparation for pansharpening

Before applying the pansharpening methods, the Sentinel-2 bands have to be upsampled to the same resolution of the PAN band, and possibly all-zero value bands have to be removed.

Open the clipped images.

In [ ]:
with rio.open(pan_path) as src:
    pan_data = src.read()
    src_meta = src.meta

In [ ]:
with rio.open(hs_path) as src:
    hs_data = src.read()

In [ ]:
band_threshold = 1e-8
hs_data = hs_data[~np.all(hs_data <= band_threshold, axis=(1,2))]

Extract the number of VNIR bands.

In [ ]:
n_bands = hs_data.shape[0]

Set the target resolution.

In [ ]:
dst_resolution = 5

Select the resampling method: this will be used to interpolate the Sentinel-2 bands to the same resolution as PAN.

In [ ]:
resampling_methods = { "nearest": Resampling.nearest,
                      "bilinear": Resampling.bilinear,
                      "cubic": Resampling.cubic,
                      "cubic_spline": Resampling.cubic_spline,
                      "lanczos": Resampling.lanczos,
                      "average": Resampling.average
}

resampling_methods_list = list(resampling_methods.keys())

resampling_w = widgets.Dropdown(
    options=resampling_methods_list,
    value='bilinear',
    description='Method:',
    disabled=False,
)
resampling_w

In [ ]:
dst_meta = src_meta.copy()
dst_meta.update({
    'height': pan_data.shape[1],
    'width': pan_data.shape[2],
    'transform': rio.Affine(dst_resolution, 0, src.bounds.left, 0, -dst_resolution, src.bounds.top),
    'dtype' : 'float32',
    'count': n_bands
})

In [ ]:
print(f"Resampling using {resampling_w.value} done")
resampled_hs = np.zeros((dst_meta['count'], src_meta['height'], src_meta['width']), dtype=hs_data.dtype)
reproject(hs_data,resampled_hs,src_transform=src.transform,src_crs=src.crs, dst_transform=dst_meta['transform'], dst_crs=src.crs, resampling=resampling_methods[resampling_w.value]);

Save the Sentinel-2 bands upsampled at 5m resolution.

In [ ]:
hs_5m_path = prisma_path + "hs_s2_5m_NN.tif"

In [ ]:
with rio.open(hs_5m_path, 'w', **dst_meta) as dst:
    dst.write(resampled_hs)

Finally, open the upsampled Sentinel-2 image at 5m resolution and remove the bands with all zeros.

In [ ]:
with rio.open(hs_5m_path) as src:
    hs_5m_data = src.read()
    
band_threshold = 1e-8
hs_5m_data = hs_5m_data[~np.all(hs_5m_data <= band_threshold, axis=(1,2))]

The number of bands without all zeros is the following:

In [ ]:
hs_5m_data.shape

Update the metadata of the image. This metadata will be used to export the pansharpened images.

In [ ]:
dst_meta.update({
    'height': int(src.height * src.transform[0] / dst_resolution),
    'width': int(src.width * src.transform[0] / dst_resolution),
    'transform': rio.Affine(dst_resolution, 0, src.bounds.left, 0, -dst_resolution, src.bounds.top),
    'dtype' : 'float32',
    'count': hs_5m_data.shape[0]
})

<div class="alert alert-info" role="alert">

## <a id='sec2'></a>&#x27A4; 2. Component Substitution Based Pansharpening

[Back to top](#TOC_TOP)

</div>

In this part of the Notebook, two Component Substitution (CS) based approaches are implemented for Sentinel-2 image pansharpening, namely the **Principal Component Analysis (PCA)** and **Gram-Schmidt (GS)** methods. An adaptive version of the GS is also implemented, namely **Gram-Schmidt Adaptive (GSA)** ([Aiazzi et al. 2007<sup>8</sup>](#8)).

These methods rely upon the projection of the higher spectral resolution image into another space, in order to separate spatial and spectral information. The transformed data are sharpened by substituting the component that containes the spatial information with the PAN image. The greater the correlation between the PAN image and the replaced component, the less spectral distortion will be introduced by the fusion approach. The fusion process is completed by applying the inverse spectral transformation to obtain the fused image.

The general equation can be expressed as follows:

$\hat{H}_k = \tilde{H}_k + G_k (P-I_L) \quad k = 1, ..., N$

* $\hat{H}_k$ is the HS pansharpened image;
* $\tilde{H}_k$ is the HS image interpolated at PAN scale;
* $P$ is the PAN image;
* $G_k$ are the gain coefficients;
* $I_L$ is the so-called intensity component;

while $k$ denotes the k-th band ($N$ is the number of bands).

$G_k$ and $I_L$ are computed differently depending on the employed method.

$I_L$ can be expressed as $I_L = \sum_{i=1}^{N} w_i \tilde{H}_i$, but different formulations exist.

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Note about the input data shape for pansharpening functions:
</div>

Input data must be provided in the following order: `(n_bands, height, width)`.

Nonetheless, in order to be saved as GeoTIFF in rasterio the order must be the following: `(height, width, n_bands)`.

### **PCA**-based pansharpening

The hypothesis underlying the application of PCA to pansharpening is that the spatial information (shared by all the channels) is concentrated in the first PC, while the spectral information specific to each single band) is accounted for the other PCs.

The vectors $w_i$ and $G_k$ of coefficient vectors are derived by the PCA procedure applied to the HS image.

In [ ]:
# Launch the algorithm
PCA_pansharpened, variance_ratios, pc_comp = pan_pca(pan_data, hs_5m_data)

The higher the explained variance of the first Principal Component, the more reliable is the method.
It is hereafter possible to display the explained variance ratio of the first 10 Principal Components.

In [ ]:
x_bar_components = 4 #Change depending on the number of PC to be plotted
pc_names = ['PC' + str(i) for i in range(1, x_bar_components+1)]

In [ ]:
plt.plot(range(x_bar_components), variance_ratios[0:x_bar_components], marker = 'o')
# set the x-axis labels
plt.xticks(range(x_bar_components), pc_names)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.show()

In [ ]:
# Save to file
with rio.open(prisma_path + 'pansharpened_s2_PCA.tif', 'w', **dst_meta) as dst:
    dst.write(PCA_pansharpened)

### **GS**-based pansharpening

The fusion process starts by using, as the component, a synthetic low resolution PAN image $I_L$ at the same spatial resolution as the HS image. A complete orthogonal decomposition is then performed, starting with that component. The pansharpening procedure is completed by substituting that component with the PAN image, and inverting the decomposition.

Gain coefficients are computed as follows:

$G_k = \frac{cov(\tilde{H}_k, I_L)}{var(I_L)}$

while the weights:

$w_i = \frac{1}{N}$

meaning that the intensity component is computed as a mean of the HS bands.

In [ ]:
# Launch the algotithm
GS_pansharpened = pan_GS(pan_data, hs_5m_data)

In [ ]:
# Save to file
with rio.open(prisma_path + 'pansharpened_s2_GS.tif', 'w', **dst_meta) as dst:
    dst.write(GS_pansharpened)

### **GSA**-based pansharpening

In adaptive version of the GS method, a linear regression between PAN and HS bands is performed. A synthetic intensity having a minimum mean-square error with respect to the reduced PAN, is computed. Accordingly, the weights $w_i$ are estimated by means of a *linear regression algorithm*.

In [ ]:
# Launch the algorithm
GSA_pansharpened = pan_GSA(pan_data, hs_data, hs_5m_data, 'local')

In [ ]:
# Save to file
with rio.open(prisma_path + 'pansharpened_s2_GSA.tif', 'w', **dst_meta) as dst:
    dst.write(GSA_pansharpened)

<div class="alert alert-info" role="alert">

## <a id='sec3'></a>&#x27A4; 3. Pansharpening Quality Assessment

[Back to top](#TOC_TOP)

</div>

### Visual inspection of the pansharpening quality

In this part of the Notebook, it is possible to export the RGB pansharpened images and inspect visually the result quality.

<div class="alert alert-success" role="alert">
<span>&#x2714;</span>
<a id='libraries'></a>
First, select which of the three pansharpened images you want to export and visualize.
</div>

In [ ]:
sw = widgets.RadioButtons(
    options=['Principal Component Analysis', 'Gram-Schmidt', 'Gram-Schmidt Adaptive'],
    description='Image to visualize',
    disabled=False,
    value='Gram-Schmidt Adaptive'
)
sw

In [ ]:
if sw.value == 'Principal Component Analysis':
    image_pansharpened = PCA_pansharpened
elif sw.value == 'Gram-Schmidt':
    image_pansharpened = GS_pansharpened
else: image_pansharpened = GSA_pansharpened

In [ ]:
# Update the number of bands to 3 before running the function
dst_meta['count'] = 3

The following function will create RGB images from the pansharpened and the original HS image. This is intended for both easier visualization in a GIS software and for convertion to PNG image.

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> In the following line of code, change the corresponding channels for red, green and blue. Default values are red=32, green=22, blue=11 for PRISMA imagery, but other values can be provided for the visualization.
</div>

In [ ]:
data_pansh, data_hs = convert_to_RGB(image_pansharpened, hs_5m_data, dst_meta, 1, 2, 3)

Export the created RGB image to JPG.

In [ ]:
matplotlib.image.imsave('img_pansh.jpg', data_pansh)
matplotlib.image.imsave('image_orig.jpg', data_hs)

Display an interactive visualization for easy comparison of the original and pansharpened images in the area of interest.

In [ ]:
img_pansh = 'img_pansh.jpg'
image_orig = "image_orig.jpg"

In [ ]:
leafmap.image_comparison(
    image_orig,
    img_pansh,
    label1='Original Image',
    label2='Pansharpened Image',
    starting_position=50,
    width=1000,
)

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> If the interactive image does not appear, run again the last code block.
</div>

### Quantitative assessment of the pansharpening quality: quality indexes computation

In this part some metrics are calculated to quantitatively assess the quality of the pansharpened images, following the reduced resolution (RR) approach.
RR approach measures the similarity of the fused product to an idea reference, namely the original HS image.
Accordingly, the resolutions of the original HS and PAN images are degradated and the fusion is performed on the degraded data.

The following quality measures are used:
1. *Spectral Angle Mapper (SAM)* that measures spectral quality;
2. *Erreur Relative Globale Adimensionnelle de Synthése (ERGAS)* which is a global adimensional quality index based on the RMSE;
3. *Spatial Correlation Coefficient (SCC)* that measure spatial quality;
4. *Peak Signal-to-Noise Ratio (PSNR)*.

Import the pansharpened images.

In [ ]:
pansharpened_img_GSA = 'pansharpened_GSA.tif'
pansharpened_img_GS = 'pansharpened_GS.tif'
pansharpened_img_PCA = 'pansharpened_PCA.tif'

In [ ]:
with rio.open(pansharpened_img_GSA) as src_pansharpened_gsa:
    # Read the data from the red, green, and blue bands
    pansharpened_gsa = src_pansharpened_gsa.read()

In [ ]:
with rio.open(pansharpened_img_GS) as src_pansharpened_gs:
    # Read the data from the red, green, and blue bands
    pansharpened_gs = src_pansharpened_gs.read()

In [ ]:
with rio.open(pansharpened_img_PCA) as src_pansharpened_pca:
    # Read the data from the red, green, and blue bands
    pansharpened_pca = src_pansharpened_pca.read()

The first step is to degrade the pansharpened image to the original HS resolution, i.e. 30 m.

In [ ]:
degrade_resolution = 30

In [ ]:
# set metadata of the degraded image
dst_meta_downgrade = dst_meta.copy()
dst_meta_downgrade.update({
    'height': int(src_pansharpened_gsa.height * src_pansharpened_gsa.transform[0] / degrade_resolution),
    'width': int(src_pansharpened_gsa.width * src_pansharpened_gsa.transform[0] / degrade_resolution),
    'transform': rio.Affine(degrade_resolution, 0, src_pansharpened_gsa.bounds.left, 0, -degrade_resolution, src_pansharpened_gsa.bounds.top),
    'dtype' : 'float32',
    'count': n_bands
})

In [ ]:
#GSA
downgraded_pansharp_gsa = np.zeros((src_pansharpened_gsa.count, dst_meta_downgrade['height'], dst_meta_downgrade['width']), dtype=pansharpened_gsa.dtype)
reproject(pansharpened_gsa,downgraded_pansharp_gsa,src_transform=src_pansharpened_gsa.transform,src_crs=src_pansharpened_gsa.crs, dst_transform=dst_meta_downgrade['transform'], 
          dst_crs=src_pansharpened_gsa.crs, resampling=Resampling.bilinear);

#GS
downgraded_pansharp_gs = np.zeros((src_pansharpened_gsa.count, dst_meta_downgrade['height'], dst_meta_downgrade['width']), dtype=pansharpened_gsa.dtype)
reproject(pansharpened_gs,downgraded_pansharp_gs,src_transform=src_pansharpened_gsa.transform,src_crs=src_pansharpened_gsa.crs, dst_transform=dst_meta_downgrade['transform'], 
          dst_crs=src_pansharpened_gsa.crs, resampling=Resampling.bilinear);

#PCA
downgraded_pansharp_pca = np.zeros((src_pansharpened_gsa.count, dst_meta_downgrade['height'], dst_meta_downgrade['width']), dtype=pansharpened_gsa.dtype)
reproject(pansharpened_pca,downgraded_pansharp_pca,src_transform=src_pansharpened_gsa.transform,src_crs=src_pansharpened_gsa.crs, dst_transform=dst_meta_downgrade['transform'], 
          dst_crs=src_pansharpened_gsa.crs, resampling=Resampling.bilinear);

Save the degraded images to GeoTIFF files.

In [ ]:
gsa_30_path = "GSA_30m.tif"
gs_30_path = "GS_30m.tif"
pca_30_path = "PCA_30m.tif"

In [ ]:
with rio.open(gsa_30_path, 'w', **dst_meta_downgrade) as dst:
    dst.write(downgraded_pansharp_gsa)

with rio.open(gs_30_path, 'w', **dst_meta_downgrade) as dst:
    dst.write(downgraded_pansharp_gs)
    
with rio.open(pca_30_path, 'w', **dst_meta_downgrade) as dst:
    dst.write(downgraded_pansharp_pca)

#### 1. Spectral Angle Mapper (SAM)

Denote ground truth by $I\in\mathbb{R}^{HW\times C}$ and generated one by $\hat{I}\in\mathbb{R}^{HW\times C}$,
SAM. $\hat{I}_i, I_i \in\mathbb{R}^{C\times 1}$ stand for the $i$-th row of $\hat{I}$ and $I$ respectively,
then

$\mathrm{SAM}(\hat{I}, I) = \frac{1}{HW}\sum_{i=1}^{HW}\arccos \frac{<\hat{I}_i, I_i>}{||\hat{I}_i||||I_i||} \,.
$

In [ ]:
print("The optimum value is 0.")
print(f"SAM applied on GSA pansharpened image: {sam(downgraded_pansharp_gsa, hs_data):.3f}")
print(f"SAM applied on GS pansharpened image: {sam(downgraded_pansharp_gs, hs_data):.3f}")
print(f"SAM applied on PCA pansharpened image: {sam(downgraded_pansharp_pca, hs_data):.3f}")

#### 2. Erreur Relative Globale Adimensionnelle de Synthése (ERGAS)

Denote the ratio of spatial resolution of PAN and LRMS by $\mathrm{scale}$ (e.g. 6 in this case, going from 5m to 30m), then

$
\mathrm{ERGAS}(\hat{I}, I) = 100*\mathrm{scale}\sqrt{\frac 1C\sum_{c=1}^C\left(\frac{\mathrm{RMSE}_c}{\mu_c}\right)^2}\,,
$

where $\mathrm{RMSE}_c$ is the RMSE between $\hat{I}_c$ and $I_c$; $\mu_c$ is mean of $I_c$.

In [ ]:
print("The optimum value is 0.")
print(f"ERGAS applied on GSA pansharpened image: {ergas(downgraded_pansharp_gsa, hs_data):.3f}")
print(f"ERGAS applied on GS pansharpened image: {ergas(downgraded_pansharp_gs, hs_data):.3f}")
print(f"ERGAS applied on PCA pansharpened image: {ergas(downgraded_pansharp_pca, hs_data):.3f}")

#### 3. Spatial Correlation Coefficient (SCC)

$\hat{I}_c, I_c \in\mathbb{R}^{HW}$ stand for the $c$-th column of $\hat{I}$ and $I$ respectively, then

$ \mathrm{SCC}(\hat{I}, I) = \frac 1C\sum_{c=1}^C\frac{\sigma_{\mathrm{cov}_c}}{\hat\sigma_c\sigma_c} $

In [ ]:
print("The optimum value is 1.")
print(f"SCC applied on GSA pansharpened image: {scc(downgraded_pansharp_gsa, hs_data):.3f}")
print(f"SCC applied on GS pansharpened image: {scc(downgraded_pansharp_gs, hs_data):.3f}")
print(f"SCC applied on PCA pansharpened image: {scc(downgraded_pansharp_pca, hs_data):.3f}")

#### 4. Peak Signal-to-Noise Ratio (PSNR)

$\mathrm{MAX}$ is the dynamic range of the pixel-values, then

$
\mathrm{PSNR}(\hat{I}, I) = 10\log_{10} \left(\frac{\mathrm{MAX}}{\mathrm{RMSE}(\hat{I}, I)}\right)^2\,.
$

In [ ]:
print("Higher PSNR is better.")
print(f"PSNR applied on GSA pansharpened image: {psnr(downgraded_pansharp_gsa, hs_data):.3f}")
print(f"PSNR applied on GS pansharpened image: {psnr(downgraded_pansharp_gs, hs_data):.3f}")
print(f"PSNR applied on PCA pansharpened image: {psnr(downgraded_pansharp_pca, hs_data):.3f}")